# Rotor 37 Tutorial
NASA Rotor 37 is a transonic axial-flow compressor rotor that has been widely studied for aerodynamic research and computational fluid dynamics (CFD) validation. Originally developed as part of a stage in a compressor used for jet engines, Rotor 37 operates at a design speed of 17,188 RPM and features a pressure ratio of about 2.1. It has 36 blades with a tip Mach number of approximately 1.38, making it ideal for studying shockwave interactions, boundary layer behavior, and overall compressor performance. The rotor has been extensively used in both experimental and numerical simulations to improve high-speed compressor designs and validate turbulence models in aerospace engineering.

[Rotor 37 Data](https://turbmodels.larc.nasa.gov/Other_exp_Data/rotor37_exp.html)

PyTurbo-Aero could be used to import the geometry for rotor 37 allowing the user to add features such as waves or shift the airfoil along the passage or even change the passage. 

In [ ]:
!pip install pyturbo-aero

In [ ]:
!git clone https://github.com/nasa/pyturbo-aero.git

In [ ]:
!cp tutorials/rotor37/* .

## Importing the Airfoil to PyTurbo-Aero

In [ ]:
# Headers 
from pyturbo.aero import Airfoil2D,Airfoil3D,Passage2D
from pyturbo.helper import bezier, bezier3, csapi
import numpy as np
import copy
import matplotlib.pyplot as plt 

### Importing the coordinates

In [ ]:
files = ['R37_profile01.csv','R37_profile02.csv','R37_profile03.csv','R37_profile04.csv','R37_profile05.csv','R37_profile06.csv']
xss = np.zeros(shape=(len(files),151)) # 5 profiles, each one is 151 points
yss = np.zeros(shape=(len(files),151))
zss = np.zeros(shape=(len(files),151))

xps = np.zeros(shape=(len(files),151))
yps = np.zeros(shape=(len(files),151))
zps = np.zeros(shape=(len(files),151))
data = list()
cx = np.zeros(shape=(len(files),1)); cy = np.zeros(shape=(len(files),1)); cz = np.zeros(shape=(len(files),1))
airfoils = list()
for i,f in enumerate(files):
    data = np.loadtxt(f,skiprows=1,delimiter=',')
    xss[i,:] = data[:151,0]
    yss[i,:] = data[:151,1]
    zss[i,:] = data[:151,2]
    xps[i,:] = np.flipud(data[150:,0])
    yps[i,:] = np.flipud(data[150:,1])
    zps[i,:] = np.flipud(data[150:,2])
    
    # Airfoil 3D - Centroid 
    cx[i] = np.hstack([xps[i,:],xss[i,:]]).mean()
    cz[i] = np.hstack([zps[i,:],zss[i,:]]).mean()
    cy[i] = np.hstack([yps[i,:],yss[i,:]]).mean()

### Creating the Airfoil 3D geometry 

In [ ]:
# Creating the Airfoil 3D 
nspan = 100
R37 = Airfoil3D([],[],0)
R37.b3 = bezier3(cx,cy,cz)
R37.shft_xss = np.zeros(shape=(nspan,151))
R37.shft_yss = np.zeros(shape=(nspan,151))

R37.shft_xps = np.zeros(shape=(nspan,151))
R37.shft_yps = np.zeros(shape=(nspan,151))

R37.shft_zss = np.zeros(shape=(nspan,151))
R37.shft_zps = np.zeros(shape=(nspan,151))

for i in range(151):        # Lets get a higher resolution airfoil
    z = np.linspace(zss[0,i],zss[-1,i],nspan)
    R37.shft_xss[:,i]= csapi(zss[:,i],xss[:,i],z)
    R37.shft_yss[:,i]= csapi(zss[:,i],yss[:,i],z)
    R37.shft_zss[:,i]= z
    
    z = np.linspace(zps[0,i],zps[-1,i],nspan)
    R37.shft_xps[:,i]= csapi(zps[:,i],xps[:,i],z)
    R37.shft_yps[:,i]= csapi(zps[:,i],yps[:,i],z)
    R37.shft_zps[:,i]= z

R37.control_x_ss = xss
R37.control_y_ss = yss
R37.control_x_ps = xps
R37.control_y_ps = yps
R37.xss = copy.deepcopy(R37.shft_xss)
R37.yss = copy.deepcopy(R37.shft_yss)
R37.xps = copy.deepcopy(R37.shft_xps)
R37.yps = copy.deepcopy(R37.shft_yps)
R37.bImportedBlade = True
R37.stackType=2 # Centroid
R37.span = max(z)-min(z)
R37.spanwise_spline_fit()
R37.nspan = nspan
R37.plot3D()

### Import the hub and shroud curves
I think the units are in inches. I don't like inches. I don't like fractions.

In [ ]:
hub = np.loadtxt('hub_R37.dat')
shroud = np.loadtxt('shroud_R37.dat')
passage = Passage2D(airfoil_array=[R37],spacing_array=[0])
passage.add_endwalls(zhub=hub[:,0],rhub=hub[:,2],zshroud=shroud[:,0],rshroud=shroud[:,2])
passage.blade_fit(hub[:,0].min())
passage.plot2D()